## Downloading the Data from WRDS and FRED

In [1]:
import pandas as pd
import numpy as np
import wrds

pd.set_option("display.width", 180)
pd.set_option("display.max_columns", 80)

db = wrds.Connection()

start_date = "2000-01-01"

# =========================
# --- S&P 500 membership ---
# =========================
sp500 = db.raw_sql(f"""
    select permno, start as begdt, ending as enddt
    from crsp.msp500list
    where coalesce(ending, date '2099-12-31') >= '{start_date}'
""")
sp500['begdt'] = pd.to_datetime(sp500['begdt'])
sp500['enddt'] = pd.to_datetime(sp500['enddt']).fillna(pd.Timestamp("2099-12-31"))

# ==================================
# --- Latest CRSP monthly/daily dt ---
# ==================================
msf_end = db.raw_sql(f"""
    select max(a.date)::date as mx
    from crsp.msf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between s.start and coalesce(s.ending, date '2099-12-31')
    where a.date >= '{start_date}'
""")['mx'].iloc[0]

dsf_end = db.raw_sql(f"""
    select max(a.date)::date as mx
    from crsp.dsf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between s.start and coalesce(s.ending, date '2099-12-31')
    where a.date >= '{start_date}'
""")['mx'].iloc[0]

# ======================
# --- CRSP monthly -----
# ======================
crsp_m = db.raw_sql(f"""
    select a.permno, a.permco, a.date::date as date,
           a.ret, a.retx,
           a.prc, a.vol, a.shrout,
           a.cfacpr, a.cfacshr
    from crsp.msf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between s.start and coalesce(s.ending, date '2099-12-31')
    where a.date between '{start_date}' and '{msf_end}'
""")
crsp_m['date'] = pd.to_datetime(crsp_m['date'])
crsp_m['prc'] = crsp_m['prc'].abs()

# ======================
# --- CRSP daily -------
# ======================
crsp_d = db.raw_sql(f"""
    select a.permno, a.date::date as date,
           a.ret, a.prc, a.vol
    from crsp.dsf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between s.start and coalesce(s.ending, date '2099-12-31')
    where a.date between '{start_date}' and '{dsf_end}'
""")
crsp_d['date'] = pd.to_datetime(crsp_d['date'])
crsp_d['prc'] = crsp_d['prc'].abs()

# ===========================================
# --- CRSP security names / industry (SIC) ---
# ===========================================
msenames = db.raw_sql(f"""
    select n.permno, n.namedt::date as namedt, n.nameendt::date as nameendt,
           n.ticker, n.shrcd, n.exchcd, n.siccd
    from crsp.msenames n
    join crsp.msp500list s
      on n.permno = s.permno
    where coalesce(n.nameendt, date '2099-12-31') >= '{start_date}'
""")
msenames['namedt'] = pd.to_datetime(msenames['namedt']).fillna(pd.Timestamp('1900-01-01'))
msenames['nameendt'] = pd.to_datetime(msenames['nameendt']).fillna(pd.Timestamp('2099-12-31'))
msenames = msenames.sort_values(['permno', 'namedt', 'nameendt'])

# =====================================
# --- CRSP/Compustat link table (CCM) ---
# =====================================
ccm = db.raw_sql("""
    select gvkey, lpermno as permno, linktype, linkprim, linkdt, linkenddt
    from crsp.ccmxpf_linktable
    where linktype in ('LU','LC') and linkprim in ('P','C')
""")
ccm['linkdt'] = pd.to_datetime(ccm['linkdt']).fillna(pd.Timestamp("1900-01-01"))
ccm['linkenddt'] = pd.to_datetime(ccm['linkenddt']).fillna(pd.Timestamp("2099-12-31"))

# ==================================================
# --- Latest FUNDQ / FUNDA dates (for PIT lagging) ---
# ==================================================
fundq_end = db.raw_sql(f"""
    select max(q.datadate)::date as mx
    from comp.fundq q
    join crsp.ccmxpf_linktable l
      on q.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
    where q.datadate between s.start and coalesce(s.ending, date '2099-12-31')
      and q.datadate >= '{start_date}'
""")['mx'].iloc[0]

funda_end = db.raw_sql(f"""
    select max(a.datadate)::date as mx
    from comp.funda a
    join crsp.ccmxpf_linktable l
      on a.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
    where a.datadate between s.start and coalesce(s.ending, date '2099-12-31')
      and a.datadate >= '{start_date}'
      and a.indfmt = 'INDL' and a.datafmt = 'STD' and a.popsrc = 'D' and a.consol = 'C'
""")['mx'].iloc[0]

# =======================================
# --- Compustat Quarterly (FUNDQ, PIT) ---
# =======================================
fundq_start = (pd.to_datetime(start_date) - pd.DateOffset(years=2)).strftime("%Y-%m-%d")
fundq = db.raw_sql(f"""
    select q.gvkey, q.datadate::date as datadate, q.fqtr, q.fyr, q.rdq,
           /* core balance sheet / income statement */
           q.atq, q.ceqq, q.cshoq, q.niq,
           q.saleq, q.cogsq, q.oibdpq, q.xsgaq,
           q.dlttq, q.dlcq, q.ltq,
           q.actq, q.cheq, q.lctq, q.dpq,
           q.revtq,
           /* cash flow & earnings */
           q.oancfy, q.ibq, q.epspxq
    from comp.fundq q
    join crsp.ccmxpf_linktable l
      on q.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
     and q.datadate between s.start and coalesce(s.ending, date '2099-12-31')
    where q.datadate between '{fundq_start}' and '{fundq_end}'
""")
fundq['datadate'] = pd.to_datetime(fundq['datadate'])

# ======================================
# --- Compustat Annual (FUNDA, PIT) ----
# ======================================
funda_start = (pd.to_datetime(start_date) - pd.DateOffset(years=2)).strftime("%Y-%m-%d")
funda = db.raw_sql(f"""
    select a.gvkey, a.datadate::date as datadate, a.fyr, a.rdp,
           a.at, a.ceq, a.seq as seq, a.txditc, a.pstkrv, a.pstkl, a.pstk,
           a.ni, a.sale, a.cogs, a.oibdp, a.xsga,
           a.dltt, a.dlc, a.lt,
           a.act, a.che, a.lct, a.dp,
           a.revt, a.ib, a.oancf
    from comp.funda a
    join crsp.ccmxpf_linktable l
      on a.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
     and a.datadate between s.start and coalesce(s.ending, date '2099-12-31')
    where a.datadate between '{funda_start}' and '{funda_end}'
      and a.indfmt = 'INDL' and a.datafmt = 'STD' and a.popsrc = 'D' and a.consol = 'C'
""")
funda['datadate'] = pd.to_datetime(funda['datadate'])

# ============================================
# --- Fama-French monthly factors (WRDS) ------
# ============================================
# This includes MKT-RF, SMB, HML, RF (pct per month)
ff = db.raw_sql(f"""
    select date::date as date, mktrf::float8 as mktrf_pct,
           smb::float8 as smb_pct, hml::float8 as hml_pct,
           rf::float8 as rf_pct
    from ff.factors_monthly
    where date >= '{start_date}'
""")
ff['date'] = pd.to_datetime(ff['date'])
ff['mktrf'] = ff['mktrf_pct'] / 100.0
ff['smb']   = ff['smb_pct']   / 100.0
ff['hml']   = ff['hml_pct']   / 100.0
ff['rf_1m'] = ff['rf_pct']    / 100.0

# ============================================
# --- CRSP Indexes for regime proxies (NEW) ---
# ============================================
# Monthly stock index (value-weighted) for trend/drawdown signals
crsp_msi = db.raw_sql(f"""
    select date::date as date, vwretd::float8 as vwretd
    from crsp.msi
    where date between '{start_date}' and '{msf_end}'
""")
crsp_msi['date'] = pd.to_datetime(crsp_msi['date'])

# Daily stock index for realized volatility
crsp_dsi = db.raw_sql(f"""
    select date::date as date, vwretd::float8 as vwretd
    from crsp.dsi
    where date between '{start_date}' and '{dsf_end}'
""")
crsp_dsi['date'] = pd.to_datetime(crsp_dsi['date'])

# --- Derived market-state features (monthly) ---
mkt = crsp_msi.sort_values('date').copy()
mkt['mkt_ret'] = mkt['vwretd']  # already decimal
# 12-month return ex-1m (classic "12-1")
mkt['mkt_ret_12m_ex1m'] = (1 + mkt['mkt_ret']).rolling(12).apply(lambda x: np.prod(x[:-1]) - 1, raw=True)
# Moving-average slope proxy (6m)
mkt['mkt_ma_6m'] = mkt['mkt_ret'].rolling(6).mean()
mkt['mkt_ma_slope_6m'] = mkt['mkt_ma_6m'] - mkt['mkt_ma_6m'].shift(3)
# Drawdown from trailing 12m high of cum product
mkt['cum'] = (1 + mkt['mkt_ret']).cumprod()
mkt['rolling_max_12m'] = mkt['cum'].rolling(12, min_periods=2).max()
mkt['drawdown_12m'] = mkt['cum'] / mkt['rolling_max_12m'] - 1.0

# Realized 1-month volatility from daily index returns (annualized)
d = crsp_dsi.sort_values('date').copy()
d['month'] = d['date'].values.astype('datetime64[M]')
realvol = (
    d.groupby('month')['vwretd']
    .apply(lambda x: np.sqrt(252) * x.std(ddof=0))
    .rename('realvol_ann')
    .reset_index()
    .rename(columns={'month':'date'})
)
realvol['date'] = pd.to_datetime(realvol['date'])

# Merge realized vol onto monthly panel
mkt = mkt.merge(realvol, on='date', how='left')

# =============================================
# --- Market-wide Amihud liquidity (NEW) -------
# =============================================
# Per stock daily Amihud = |ret| / dollar_vol; monthly average per stock; cross-sec median each month
crsp_d['dollar_vol'] = (crsp_d['vol'].astype(float) * crsp_d['prc'].astype(float)).replace([np.inf, -np.inf], np.nan)
# Filter impossible values
valid = crsp_d[['permno','date','ret','dollar_vol']].dropna()
valid = valid[(valid['dollar_vol'] > 0) & valid['ret'].notna()]
valid['amihud'] = np.abs(valid['ret'].astype(float)) / valid['dollar_vol'].astype(float)
valid['month']  = valid['date'].values.astype('datetime64[M]')
ami_stock_m = valid.groupby(['permno','month'])['amihud'].mean().reset_index().rename(columns={'month':'date'})
ami_mkt = ami_stock_m.groupby('date')['amihud'].median().reset_index().rename(columns={'amihud':'amihud_median'})

# Merge Amihud into market panel
mkt = mkt.merge(ami_mkt, on='date', how='left')

# =============================================
# --- OPTIONAL: FRED macro series (NEW) -------
# =============================================
# These are helpful for a "risk-off" HMM. Wrapped in try/except in case FRED is not enabled on your WRDS.
def try_fred(series_id):
    q = f"""
        select date::date as date, value::float8 as value
        from fred.fred_series_observations
        where series_id = '{series_id}' and date >= '{start_date}'
    """
    try:
        tmp = db.raw_sql(q)
        tmp['date'] = pd.to_datetime(tmp['date'])
        return tmp.rename(columns={'value': series_id})
    except Exception as e:
        print(f"[FRED unavailable or series not found] {series_id}: {e}")
        return None

fred_series = {
    'T10Y2Y':    'Term Spread (10y-2y, %)',
    'BAA10Y':    'Moody Baa – 10y Treasury (% pts)',
    'VIXCLS':    'CBOE VIX (level)',
    'TEDRATE':   'TED Spread (%)'
}
fred_list = [try_fred(s) for s in fred_series.keys()]
fred_list = [x for x in fred_list if x is not None]

if len(fred_list) > 0:
    fred = fred_list[0]
    for x in fred_list[1:]:
        fred = fred.merge(x, on='date', how='outer')
    fred = fred.sort_values('date')
else:
    fred = pd.DataFrame(columns=['date'])

# Merge FRED onto market panel (align monthly by end-of-month)
if not fred.empty:
    fred_m = fred.copy()
    fred_m['date'] = fred_m['date'].values.astype('datetime64[M]') + np.timedelta64(1, 'M') - np.timedelta64(1, 'D')
    fred_m = fred_m.groupby('date').last().reset_index()
    mkt = mkt.merge(fred_m, on='date', how='left')

# =============================================
# --- Final regime-feature panel (monthly) -----
# =============================================
regime_features = (
    mkt[['date', 'mkt_ret', 'mkt_ret_12m_ex1m', 'mkt_ma_slope_6m',
         'drawdown_12m', 'realvol_ann', 'amihud_median']]
    .merge(ff[['date','mktrf','smb','hml','rf_1m']], on='date', how='left')
    .sort_values('date')
    .reset_index(drop=True)
)

# ================================
# --- Light sanity adjustments ----
# ================================
for df in (crsp_m, crsp_d):
    if 'shrout' in df.columns:
        df['shrout'] = pd.to_numeric(df['shrout'], errors='coerce')
    df['prc'] = pd.to_numeric(df['prc'], errors='coerce').abs()
    if 'vol' in df.columns:
        df['dollar_vol'] = (df['vol'].astype(float) * df['prc'].astype(float)).replace([np.inf, -np.inf], np.nan)

# =========================
# --- Diagnostics print ----
# =========================
print("Latest available dates:")
print(f"CRSP monthly: {msf_end}")
print(f"CRSP daily  : {dsf_end}")
print(f"FUNDQ end   : {fundq_end}")
print(f"FUNDA end   : {funda_end}")
print(f"FF factors  : {ff['date'].max().date()}")

print("Data shapes:")
print("sp500           ", sp500.shape)
print("crsp_m          ", crsp_m.shape)
print("crsp_d          ", crsp_d.shape)
print("msenames        ", msenames.shape)
print("ccm             ", ccm.shape)
print("fundq           ", fundq.shape)
print("funda           ", funda.shape)
print("ff factors      ", ff.shape)
print("crsp_msi (mkt)  ", crsp_msi.shape)
print("crsp_dsi (mkt)  ", crsp_dsi.shape)
print("regime_features ", regime_features.shape)
if not fred.empty:
    print("FRED merged     ", fred.shape)
else:
    print("FRED merged     : none (skipped)")


Enter your WRDS username [woota]: chriswt
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
[FRED unavailable or series not found] T10Y2Y: (psycopg2.errors.UndefinedTable) relation "fred.fred_series_observations" does not exist
LINE 3:         from fred.fred_series_observations
                     ^

[SQL: 
        select date::date as date, value::float8 as value
        from fred.fred_series_observations
        where series_id = 'T10Y2Y' and date >= '2010-01-01'
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)
[FRED unavailable or series not found] BAA10Y: (psycopg2.errors.UndefinedTable) relation "fred.fred_series_observations" does not exist
LINE 3:         from fred.fred_series_observations
                     ^

[SQL: 
        select date::date as date, value::float8 as value
        from fred.fred_series_observations
        where series_id = 'BAA10Y' and date >= '2010-01-01'
    ]
(Background on this

In [2]:
import pandas as pd
import numpy as np
import wrds

pd.set_option("display.width", 180)
pd.set_option("display.max_columns", 80)

db = wrds.Connection()

# -------------------------------------------------
# Pull window: 2000-01-01 to the most recent data
# -------------------------------------------------
start_date = "2000-01-01"

# =========================
# --- S&P 500 membership ---
# =========================
# Keep any membership spell that overlaps 2000+.
sp500 = db.raw_sql(f"""
    select permno, start as begdt, ending as enddt
    from crsp.msp500list
    where coalesce(ending, date '2099-12-31') >= '{start_date}'
""")
sp500['begdt'] = pd.to_datetime(sp500['begdt'])
sp500['enddt'] = pd.to_datetime(sp500['enddt']).fillna(pd.Timestamp("2099-12-31"))

# ==================================
# --- Latest CRSP monthly/daily dt ---
# ==================================
# Compute end dates using 2000+ overlaps only.
msf_end = db.raw_sql(f"""
    select max(a.date)::date as mx
    from crsp.msf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between greatest(s.start, date '{start_date}')
                     and coalesce(s.ending, date '2099-12-31')
    where a.date >= '{start_date}'
""")['mx'].iloc[0]

dsf_end = db.raw_sql(f"""
    select max(a.date)::date as mx
    from crsp.dsf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between greatest(s.start, date '{start_date}')
                     and coalesce(s.ending, date '2099-12-31')
    where a.date >= '{start_date}'
""")['mx'].iloc[0]

# ======================
# --- CRSP monthly -----
# ======================
crsp_m = db.raw_sql(f"""
    select a.permno, a.permco, a.date::date as date,
           a.ret, a.retx,
           a.prc, a.vol, a.shrout,
           a.cfacpr, a.cfacshr
    from crsp.msf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between greatest(s.start, date '{start_date}')
                     and coalesce(s.ending, date '2099-12-31')
    where a.date between '{start_date}' and '{msf_end}'
""")
crsp_m['date'] = pd.to_datetime(crsp_m['date'])
crsp_m['prc'] = crsp_m['prc'].abs()

# ======================
# --- CRSP daily -------
# ======================
crsp_d = db.raw_sql(f"""
    select a.permno, a.date::date as date,
           a.ret, a.prc, a.vol
    from crsp.dsf a
    join crsp.msp500list s
      on a.permno = s.permno
     and a.date between greatest(s.start, date '{start_date}')
                     and coalesce(s.ending, date '2099-12-31')
    where a.date between '{start_date}' and '{dsf_end}'
""")
crsp_d['date'] = pd.to_datetime(crsp_d['date'])
crsp_d['prc'] = crsp_d['prc'].abs()

# ===========================================
# --- CRSP security names / industry (SIC) ---
# ===========================================
# Include name histories that are active at any time since 2000.
msenames = db.raw_sql(f"""
    select n.permno, n.namedt::date as namedt, n.nameendt::date as nameendt,
           n.ticker, n.shrcd, n.exchcd, n.siccd
    from crsp.msenames n
    join crsp.msp500list s
      on n.permno = s.permno
    where coalesce(n.nameendt, date '2099-12-31') >= '{start_date}'
""")
msenames['namedt'] = pd.to_datetime(msenames['namedt']).fillna(pd.Timestamp('1900-01-01'))
msenames['nameendt'] = pd.to_datetime(msenames['nameendt']).fillna(pd.Timestamp('2099-12-31'))
msenames = msenames.sort_values(['permno', 'namedt', 'nameendt'])

# =====================================
# --- CRSP/Compustat link table (CCM) ---
# =====================================
# Keep standard link filters; do NOT force linkdt >= 2000
# so that pre-2000 links that extend into 2000+ are retained.
ccm = db.raw_sql("""
    select gvkey, lpermno as permno, linktype, linkprim, linkdt, linkenddt
    from crsp.ccmxpf_linktable
    where linktype in ('LU','LC') and linkprim in ('P','C')
""")
ccm['linkdt'] = pd.to_datetime(ccm['linkdt']).fillna(pd.Timestamp("1900-01-01"))
ccm['linkenddt'] = pd.to_datetime(ccm['linkenddt']).fillna(pd.Timestamp("2099-12-31"))

# ==================================================
# --- Latest FUNDQ / FUNDA dates (for PIT lagging) ---
# ==================================================
# Again, ensure overlaps are 2000+ via greatest(s.start, start_date).
fundq_end = db.raw_sql(f"""
    select max(q.datadate)::date as mx
    from comp.fundq q
    join crsp.ccmxpf_linktable l
      on q.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
    where q.datadate between greatest(s.start, date '{start_date}')
                         and coalesce(s.ending, date '2099-12-31')
      and q.datadate >= '{start_date}'
""")['mx'].iloc[0]

funda_end = db.raw_sql(f"""
    select max(a.datadate)::date as mx
    from comp.funda a
    join crsp.ccmxpf_linktable l
      on a.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
    where a.datadate between greatest(s.start, date '{start_date}')
                         and coalesce(s.ending, date '2099-12-31')
      and a.datadate >= '{start_date}'
      and a.indfmt = 'INDL' and a.datafmt = 'STD' and a.popsrc = 'D' and a.consol = 'C'
""")['mx'].iloc[0]

# =======================================
# --- Compustat Quarterly (FUNDQ, PIT) ---
# =======================================
# Pull a 2-year lookback to compute PIT lags properly, but
# still cap joins to membership overlapping 2000+.
fundq_start = (pd.to_datetime(start_date) - pd.DateOffset(years=2)).strftime("%Y-%m-%d")
fundq = db.raw_sql(f"""
    select q.gvkey, q.datadate::date as datadate, q.fqtr, q.fyr, q.rdq,
           /* core balance sheet / income statement */
           q.atq, q.ceqq, q.cshoq, q.niq,
           q.saleq, q.cogsq, q.oibdpq, q.xsgaq,
           q.dlttq, q.dlcq, q.ltq,
           q.actq, q.cheq, q.lctq, q.dpq,
           q.revtq,
           /* cash flow & earnings */
           q.oancfy, q.ibq, q.epspxq
    from comp.fundq q
    join crsp.ccmxpf_linktable l
      on q.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
     and q.datadate between greatest(s.start, date '{start_date}')
                         and coalesce(s.ending, date '2099-12-31')
    where q.datadate between '{fundq_start}' and '{fundq_end}'
""")
fundq['datadate'] = pd.to_datetime(fundq['datadate'])

# ======================================
# --- Compustat Annual (FUNDA, PIT) ----
# ======================================
funda_start = (pd.to_datetime(start_date) - pd.DateOffset(years=2)).strftime("%Y-%m-%d")
funda = db.raw_sql(f"""
    select a.gvkey, a.datadate::date as datadate, a.fyr, a.rdp,
           a.at, a.ceq, a.seq as seq, a.txditc, a.pstkrv, a.pstkl, a.pstk,
           a.ni, a.sale, a.cogs, a.oibdp, a.xsga,
           a.dltt, a.dlc, a.lt,
           a.act, a.che, a.lct, a.dp,
           a.revt, a.ib, a.oancf
    from comp.funda a
    join crsp.ccmxpf_linktable l
      on a.gvkey = l.gvkey
     and l.linktype in ('LU','LC') and l.linkprim in ('P','C')
    join crsp.msp500list s
      on l.lpermno = s.permno
     and a.datadate between greatest(s.start, date '{start_date}')
                         and coalesce(s.ending, date '2099-12-31')
    where a.datadate between '{funda_start}' and '{funda_end}'
      and a.indfmt = 'INDL' and a.datafmt = 'STD' and a.popsrc = 'D' and a.consol = 'C'
""")
funda['datadate'] = pd.to_datetime(funda['datadate'])

# ============================================
# --- Fama-French monthly factors (WRDS) ------
# ============================================
# This includes MKT-RF, SMB, HML, RF (pct per month)
ff = db.raw_sql(f"""
    select date::date as date, mktrf::float8 as mktrf_pct,
           smb::float8 as smb_pct, hml::float8 as hml_pct,
           rf::float8 as rf_pct
    from ff.factors_monthly
    where date >= '{start_date}'
""")
ff['date'] = pd.to_datetime(ff['date'])
ff['mktrf'] = ff['mktrf_pct'] / 100.0
ff['smb']   = ff['smb_pct']   / 100.0
ff['hml']   = ff['hml_pct']   / 100.0
ff['rf_1m'] = ff['rf_pct']    / 100.0

# ============================================
# --- CRSP Indexes for regime proxies (NEW) ---
# ============================================
# Monthly stock index (value-weighted) for trend/drawdown signals
crsp_msi = db.raw_sql(f"""
    select date::date as date, vwretd::float8 as vwretd
    from crsp.msi
    where date between '{start_date}' and '{msf_end}'
""")
crsp_msi['date'] = pd.to_datetime(crsp_msi['date'])

# Daily stock index for realized volatility
crsp_dsi = db.raw_sql(f"""
    select date::date as date, vwretd::float8 as vwretd
    from crsp.dsi
    where date between '{start_date}' and '{dsf_end}'
""")
crsp_dsi['date'] = pd.to_datetime(crsp_dsi['date'])

# --- Derived market-state features (monthly) ---
mkt = crsp_msi.sort_values('date').copy()
mkt['mkt_ret'] = mkt['vwretd']  # already decimal
# 12-month return ex-1m (classic "12-1")
mkt['mkt_ret_12m_ex1m'] = (1 + mkt['mkt_ret']).rolling(12).apply(lambda x: np.prod(x[:-1]) - 1, raw=True)
# Moving-average slope proxy (6m)
mkt['mkt_ma_6m'] = mkt['mkt_ret'].rolling(6).mean()
mkt['mkt_ma_slope_6m'] = mkt['mkt_ma_6m'] - mkt['mkt_ma_6m'].shift(3)
# Drawdown from trailing 12m high of cum product
mkt['cum'] = (1 + mkt['mkt_ret']).cumprod()
mkt['rolling_max_12m'] = mkt['cum'].rolling(12, min_periods=2).max()
mkt['drawdown_12m'] = mkt['cum'] / mkt['rolling_max_12m'] - 1.0

# Realized 1-month volatility from daily index returns (annualized)
d = crsp_dsi.sort_values('date').copy()
d['month'] = d['date'].values.astype('datetime64[M]')
realvol = (
    d.groupby('month')['vwretd']
    .apply(lambda x: np.sqrt(252) * x.std(ddof=0))
    .rename('realvol_ann')
    .reset_index()
    .rename(columns={'month':'date'})
)
realvol['date'] = pd.to_datetime(realvol['date'])

# Merge realized vol onto monthly panel
mkt = mkt.merge(realvol, on='date', how='left')

# =============================================
# --- Market-wide Amihud liquidity (NEW) -------
# =============================================
# Per stock daily Amihud = |ret| / dollar_vol; monthly average per stock; cross-sec median each month
crsp_d['dollar_vol'] = (crsp_d['vol'].astype(float) * crsp_d['prc'].astype(float)).replace([np.inf, -np.inf], np.nan)
# Filter impossible values
valid = crsp_d[['permno','date','ret','dollar_vol']].dropna()
valid = valid[(valid['dollar_vol'] > 0) & valid['ret'].notna()]
valid['amihud'] = np.abs(valid['ret'].astype(float)) / valid['dollar_vol'].astype(float)
valid['month']  = valid['date'].values.astype('datetime64[M]')
ami_stock_m = valid.groupby(['permno','month'])['amihud'].mean().reset_index().rename(columns={'month':'date'})
ami_mkt = ami_stock_m.groupby('date')['amihud'].median().reset_index().rename(columns={'amihud':'amihud_median'})

# Merge Amihud into market panel
mkt = mkt.merge(ami_mkt, on='date', how='left')

# =============================================
# --- OPTIONAL: FRED macro series (NEW) -------
# =============================================
# These are helpful for a "risk-off" HMM. Wrapped in try/except in case FRED is not enabled on your WRDS.
def try_fred(series_id):
    q = f"""
        select date::date as date, value::float8 as value
        from fred.fred_series_observations
        where series_id = '{series_id}' and date >= '{start_date}'
    """
    try:
        tmp = db.raw_sql(q)
        tmp['date'] = pd.to_datetime(tmp['date'])
        return tmp.rename(columns={'value': series_id})
    except Exception as e:
        print(f"[FRED unavailable or series not found] {series_id}: {e}")
        return None

fred_series = {
    'T10Y2Y':    'Term Spread (10y-2y, %)',
    'BAA10Y':    'Moody Baa – 10y Treasury (% pts)',
    'VIXCLS':    'CBOE VIX (level)',
    'TEDRATE':   'TED Spread (%)'
}
fred_list = [try_fred(s) for s in fred_series.keys()]
fred_list = [x for x in fred_list if x is not None]

if len(fred_list) > 0:
    fred = fred_list[0]
    for x in fred_list[1:]:
        fred = fred.merge(x, on='date', how='outer')
    fred = fred.sort_values('date')
else:
    fred = pd.DataFrame(columns=['date'])

# Merge FRED onto market panel (align monthly by end-of-month)
if not fred.empty:
    fred_m = fred.copy()
    fred_m['date'] = fred_m['date'].values.astype('datetime64[M]') + np.timedelta64(1, 'M') - np.timedelta64(1, 'D')
    fred_m = fred_m.groupby('date').last().reset_index()
    mkt = mkt.merge(fred_m, on='date', how='left')

# =============================================
# --- Final regime-feature panel (monthly) -----
# =============================================
regime_features = (
    mkt[['date', 'mkt_ret', 'mkt_ret_12m_ex1m', 'mkt_ma_slope_6m',
         'drawdown_12m', 'realvol_ann', 'amihud_median']]
    .merge(ff[['date','mktrf','smb','hml','rf_1m']], on='date', how='left')
    .sort_values('date')
    .reset_index(drop=True)
)

# ================================
# --- Light sanity adjustments ----
# ================================
for df in (crsp_m, crsp_d):
    if 'shrout' in df.columns:
        df['shrout'] = pd.to_numeric(df['shrout'], errors='coerce')
    df['prc'] = pd.to_numeric(df['prc'], errors='coerce').abs()
    if 'vol' in df.columns:
        df['dollar_vol'] = (df['vol'].astype(float) * df['prc'].astype(float)).replace([np.inf, -np.inf], np.nan)

# =========================
# --- Diagnostics print ----
# =========================
print("Latest available dates:")
print(f"CRSP monthly: {msf_end}")
print(f"CRSP daily  : {dsf_end}")
print(f"FUNDQ end   : {fundq_end}")
print(f"FUNDA end   : {funda_end}")
print(f"FF factors  : {ff['date'].max().date()}")

print("Data shapes:")
print("sp500           ", sp500.shape)
print("crsp_m          ", crsp_m.shape)
print("crsp_d          ", crsp_d.shape)
print("msenames        ", msenames.shape)
print("ccm             ", ccm.shape)
print("fundq           ", fundq.shape)
print("funda           ", funda.shape)
print("ff factors      ", ff.shape)
print("crsp_msi (mkt)  ", crsp_msi.shape)
print("crsp_dsi (mkt)  ", crsp_dsi.shape)
print("regime_features ", regime_features.shape)
if not fred.empty:
    print("FRED merged     ", fred.shape)
else:
    print("FRED merged     : none (skipped)")


Enter your WRDS username [woota]: chriswt
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
[FRED unavailable or series not found] T10Y2Y: (psycopg2.errors.UndefinedTable) relation "fred.fred_series_observations" does not exist
LINE 3:         from fred.fred_series_observations
                     ^

[SQL: 
        select date::date as date, value::float8 as value
        from fred.fred_series_observations
        where series_id = 'T10Y2Y' and date >= '2000-01-01'
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)
[FRED unavailable or series not found] BAA10Y: (psycopg2.errors.UndefinedTable) relation "fred.fred_series_observations" does not exist
LINE 3:         from fred.fred_series_observations
                     ^

[SQL: 
        select date::date as date, value::float8 as value
        from fred.fred_series_observations
        where series_id = 'BAA10Y' and date >= '2000-01-01'
    ]
(Background on this

## Merging Dataframes

In [3]:
import pandas as pd
import numpy as np

# ============================================================
# FACTOR-MODEL MERGE (robust; no merge_asof; 2m PIT lag)
# Inputs expected in memory: crsp_m, crsp_d, ccm, fundq, funda
# Also need risk-free: either `rf` (with rf_1m) or `ff` (with rf_1m)
# Optional: sp500, msenames
# Output: factor_base (monthly panel for factor model)
# ============================================================

# ---------- helpers ----------
def safe_num(s):
    return pd.to_numeric(s, errors='coerce')

def compustat_avail_month(date_series, lag_months=2):
    d = pd.to_datetime(date_series).dt.tz_localize(None)
    return (d + pd.offsets.MonthEnd(lag_months)).dt.to_period('M')

def safe_daily_std(x):
    """std of daily returns within a month without warnings; NaN if <2 obs."""
    arr = pd.Series(x, dtype='float64').dropna().to_numpy()
    if arr.size < 2:
        return np.nan
    return np.std(arr, ddof=0)  # population std typical for realized vol

def coalesce(*series):
    out = series[0].copy()
    for s in series[1:]:
        m = out.isna()
        out[m] = s[m]
    return out

# ---------- 0) CRSP monthly ----------
crsp_m = crsp_m.copy()
crsp_m['date']   = pd.to_datetime(crsp_m['date']).dt.tz_localize(None)
crsp_m['prc']    = safe_num(crsp_m['prc']).abs()
crsp_m['shrout'] = safe_num(crsp_m['shrout'])
crsp_m['ret']    = safe_num(crsp_m['ret'])
crsp_m['retx']   = safe_num(crsp_m['retx'])
crsp_m['vol']    = safe_num(crsp_m['vol'])
crsp_m['mktcap'] = crsp_m['prc'] * crsp_m['shrout'] / 1000.0   # $mm (shrout in thousands)
crsp_m['month']  = crsp_m['date'].dt.to_period('M')
crsp_m = crsp_m.sort_values(['permno','date']).drop_duplicates(['permno','date'], keep='last')

# ---------- 1) CRSP daily -> monthly stats ----------
crsp_d = crsp_d.copy()
crsp_d['date']  = pd.to_datetime(crsp_d['date']).dt.tz_localize(None)
crsp_d['prc']   = safe_num(crsp_d['prc']).abs()
crsp_d['vol']   = safe_num(crsp_d['vol'])
crsp_d['ret']   = safe_num(crsp_d['ret'])
crsp_d['month'] = crsp_d['date'].dt.to_period('M')
crsp_d['dollar_vol'] = crsp_d['prc'] * crsp_d['vol']

daily_stats = (
    crsp_d.groupby(['permno','month'], as_index=False)
          .agg(
              volatility_daily=('ret', safe_daily_std),
              avg_dollar_vol=('dollar_vol','mean')
          )
)

# ---------- 2) merge daily stats ----------
crsp_all = crsp_m.merge(daily_stats, on=['permno','month'], how='left')

# ---------- 3) optional: S&P 500 PIT filter ----------
if 'sp500' in globals():
    sp500 = sp500.copy()
    sp500['begdt'] = pd.to_datetime(sp500['begdt']).dt.tz_localize(None)
    sp500['enddt'] = pd.to_datetime(sp500['enddt']).dt.tz_localize(None)
    sp500['beg_m'] = sp500['begdt'].dt.to_period('M')
    sp500['end_m'] = sp500['enddt'].dt.to_period('M')
    crsp_all = crsp_all.merge(sp500[['permno','beg_m','end_m']], on='permno', how='left')
    crsp_all = crsp_all[(crsp_all['month'] >= crsp_all['beg_m']) & (crsp_all['month'] <= crsp_all['end_m'])].drop(columns=['beg_m','end_m'])

# ---------- 4) optional: ticker/SIC from msenames ----------
if 'msenames' in globals():
    mn = msenames.copy()
    mn['namedt']   = pd.to_datetime(mn['namedt']).dt.tz_localize(None)
    mn['nameendt'] = pd.to_datetime(mn['nameendt']).dt.tz_localize(None).fillna(pd.Timestamp('2099-12-31'))
    mn['start_m']  = mn['namedt'].dt.to_period('M')
    mn['end_m']    = mn['nameendt'].dt.to_period('M')
    mn = mn[['permno','ticker','shrcd','exchcd','siccd','start_m','end_m']].dropna(subset=['start_m'])
    mn['end_m']    = mn['end_m'].fillna(pd.Period('2099-12', freq='M'))
    mn['month']    = mn.apply(lambda r: pd.period_range(r['start_m'], r['end_m'], freq='M'), axis=1)
    mn = mn.explode('month').drop(columns=['start_m','end_m']).drop_duplicates(['permno','month'])
    crsp_all = crsp_all.merge(mn, on=['permno','month'], how='left')

# ---------- 5) CCM link (and window) ----------
ccm = ccm.copy()
ccm['linkdt']    = pd.to_datetime(ccm['linkdt']).dt.tz_localize(None).fillna(pd.Timestamp('1900-01-01'))
ccm['linkenddt'] = pd.to_datetime(ccm['linkenddt']).dt.tz_localize(None).fillna(pd.Timestamp('2099-12-31'))
ccm['start_m']   = ccm['linkdt'].dt.to_period('M')
ccm['end_m']     = ccm['linkenddt'].dt.to_period('M')
ccm['gvkey']     = ccm['gvkey'].astype(str)

crsp_all = crsp_all.merge(ccm[['gvkey','permno','start_m','end_m']], on='permno', how='left')
crsp_all = crsp_all.dropna(subset=['gvkey'])
crsp_all = crsp_all[(crsp_all['month'] >= crsp_all['start_m']) & (crsp_all['month'] <= crsp_all['end_m'])].drop(columns=['start_m','end_m'])
crsp_all['gvkey'] = crsp_all['gvkey'].astype(str)

# ---------- 6) FUNDQ with 2‑month PIT lag, then ffill by gvkey ----------
fq = fundq.copy()
fq['gvkey']    = fq['gvkey'].astype(str)
fq['datadate'] = pd.to_datetime(fq['datadate']).dt.tz_localize(None)
fq['rdq']      = pd.to_datetime(fq.get('rdq')).dt.tz_localize(None)
fq['avail_m']  = compustat_avail_month(coalesce(fq['rdq'], fq['datadate']), lag_months=2)
fq = fq.sort_values(['gvkey','avail_m','datadate']).drop_duplicates(['gvkey','avail_m'], keep='last')

grid_q = crsp_all[['gvkey','month']].drop_duplicates().sort_values(['gvkey','month'])
fq_for_merge = fq.copy()
fq_for_merge['month'] = fq_for_merge['avail_m']
fq_for_merge = fq_for_merge.drop(columns=['avail_m'])
fq_cols = [c for c in fq_for_merge.columns if c not in ['gvkey','month']]
fund_on_grid = grid_q.merge(fq_for_merge[['gvkey','month'] + fq_cols], on=['gvkey','month'], how='left')
fund_on_grid[fq_cols] = fund_on_grid.groupby('gvkey')[fq_cols].ffill()

# ---------- 7) FUNDA with 2‑month PIT lag, then ffill by gvkey ----------
fa = funda.copy()
fa['gvkey']    = fa['gvkey'].astype(str)
fa['datadate'] = pd.to_datetime(fa['datadate']).dt.tz_localize(None)
fa['rdp']      = pd.to_datetime(fa.get('rdp')).dt.tz_localize(None)
fa['avail_m']  = compustat_avail_month(coalesce(fa['rdp'], fa['datadate']), lag_months=2)
fa = fa.sort_values(['gvkey','avail_m','datadate']).drop_duplicates(['gvkey','avail_m'], keep='last')

grid_a = crsp_all[['gvkey','month']].drop_duplicates().sort_values(['gvkey','month'])
fa_for_merge = fa.copy()
fa_for_merge['month'] = fa_for_merge['avail_m']
fa_for_merge = fa_for_merge.drop(columns=['avail_m'])
fa_cols = [c for c in fa_for_merge.columns if c not in ['gvkey','month']]
fundA_on_grid = grid_a.merge(fa_for_merge[['gvkey','month'] + fa_cols], on=['gvkey','month'], how='left')
fundA_on_grid[fa_cols] = fundA_on_grid.groupby('gvkey')[fa_cols].ffill()

# ---------- 8) join Compustat (Q & A) + RF ----------
final = crsp_all.merge(fund_on_grid, on=['gvkey','month'], how='left', suffixes=('','_q'))
final = final.merge(fundA_on_grid, on=['gvkey','month'], how='left', suffixes=('','_a'))

# Risk-free: accept either rf or ff in globals
if 'rf' in globals():
    rf_m = rf.copy()
    rf_m['date']  = pd.to_datetime(rf_m['date']).dt.tz_localize(None)
    rf_m['month'] = rf_m['date'].dt.to_period('M')
    rf_m = rf_m.drop_duplicates('month')[['month','rf_1m']]
elif 'ff' in globals():
    rf_m = ff.copy()
    rf_m['date']  = pd.to_datetime(ff['date']).dt.tz_localize(None)
    rf_m['month'] = rf_m['date'].dt.to_period('M')
    rf_m = rf_m.drop_duplicates('month')[['month','rf_1m']]
else:
    raise RuntimeError("Missing risk-free series: provide `rf` (WRDS FF) or `ff` with column `rf_1m`.")

final = final.merge(rf_m, on='month', how='left').sort_values(['permno','month'])

# ---------- 9) targets + lagged CRSP features ----------
final['ret_fwd_1m']        = final.groupby('permno')['ret'].shift(-1)
final['excess_ret_fwd_1m'] = final['ret_fwd_1m'] - final['rf_1m']

final['mktcap_lag1']          = final.groupby('permno')['mktcap'].shift(1)
final['avg_dollar_vol_lag1']  = final.groupby('permno')['avg_dollar_vol'].shift(1)

# ---------- 10) select output ----------
fq_keep = ['atq','ceqq','cshoq','niq','saleq','cogsq','oibdpq','xsgaq',
           'dlttq','dlcq','ltq','actq','cheq','lctq','dpq','revtq','oancfy','ibq','epspxq','fqtr','fyr','datadate']
fa_keep = ['at','ceq','seq','txditc','pstkrv','pstkl','pstk','ni','sale','cogs','oibdp','xsga',
           'dltt','dlc','lt','act','che','lct','dp','revt','ib','oancf','fyr','datadate']

cols_out = [
    'permno','gvkey','month','date','ticker','shrcd','exchcd','siccd',
    'prc','shrout','mktcap','mktcap_lag1',
    'vol','volatility_daily','avg_dollar_vol','avg_dollar_vol_lag1',
    'ret','retx','rf_1m','ret_fwd_1m','excess_ret_fwd_1m'
] + [c for c in fq_keep if c in final.columns] + [c for c in fa_keep if c in final.columns]

factor_base = final[cols_out].sort_values(['permno','month']).reset_index(drop=True)

print("factor_base shape:", factor_base.shape)
print(factor_base.head(3))

# ---------- PLACEHOLDER (later) ----------
# After you train your HMMs, create `regime_probs` with columns ['date','p_riskoff','p_trend'] (monthly)
# and merge onto factor_base by month:
# rp = regime_probs.copy()
# rp['date']  = pd.to_datetime(rp['date']).dt.tz_localize(None)
# rp['month'] = rp['date'].dt.to_period('M')
# rp = rp.drop_duplicates('month')[['month','p_riskoff','p_trend']]
# factor_base = factor_base.merge(rp, on='month', how='left')

factor_base shape: (149669, 67)
   permno   gvkey    month       date ticker  shrcd  exchcd  siccd       prc     shrout         mktcap    mktcap_lag1        vol  volatility_daily     avg_dollar_vol  \
0   10078  012136  2000-01 2000-01-31   SUNW     11       3   3570   78.5625  1561106.0  122644.390125           <NA>  4163042.0          0.044158   1619769833.64375   
1   10078  012136  2000-02 2000-02-29   SUNW     11       3   3570     95.25  1561106.0    148695.3465  122644.390125  3131641.0          0.031544    1419137165.2125   
2   10078  012136  2000-03 2000-03-31   SUNW     11       3   3570  93.70313  1586413.0  148651.863573    148695.3465  3521264.0          0.033648  1463410044.721832   

   avg_dollar_vol_lag1       ret      retx     rf_1m  ret_fwd_1m  excess_ret_fwd_1m   atq  ceqq  cshoq   niq  saleq  cogsq  oibdpq  xsgaq  dlttq  dlcq   ltq  actq  cheq  lctq  \
0                 <NA>  0.014528  0.014528  0.000041     0.21241           0.212369  <NA>  <NA>   <NA>  <NA>   <N

In [7]:
factor_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149669 entries, 0 to 149668
Data columns (total 67 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   permno               149669 non-null  Int64         
 1   gvkey                149669 non-null  object        
 2   month                149669 non-null  period[M]     
 3   date                 149669 non-null  datetime64[ns]
 4   ticker               149669 non-null  string        
 5   shrcd                149669 non-null  Int64         
 6   exchcd               149669 non-null  Int64         
 7   siccd                149669 non-null  Int64         
 8   prc                  149661 non-null  Float64       
 9   shrout               149669 non-null  Float64       
 10  mktcap               149661 non-null  Float64       
 11  mktcap_lag1          148616 non-null  Float64       
 12  vol                  149669 non-null  Float64       
 13  volatility_dai

In [8]:
factor_base.drop('datadate', axis=1) 

,permno,gvkey,month,date,ticker,shrcd,exchcd,siccd,prc,shrout,mktcap,mktcap_lag1,vol,volatility_daily,avg_dollar_vol,avg_dollar_vol_lag1,ret,retx,rf_1m,ret_fwd_1m,excess_ret_fwd_1m,atq,ceqq,cshoq,niq,saleq,cogsq,oibdpq,xsgaq,dlttq,dlcq,ltq,actq,cheq,lctq,dpq,revtq,oancfy,ibq,epspxq,fqtr,fyr,at,ceq,seq,txditc,pstkrv,pstkl,pstk,ni,sale,cogs,oibdp,xsga,dltt,dlc,lt,act,che,lct,dp,revt,ib,oancf,fyr
0,10078,012136,2000-01,2000-01-31,SUNW,11,3,3570,78.5625,1561106.0,122644.390125,<NA>,4163042.0,0.044158,1619769833.64375,<NA>,0.014528,0.014528,0.000041,0.21241,0.212369,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,10078,012136,2000-02,2000-02-29,SUNW,11,3,3570,95.25,1561106.0,148695.3465,122644.390125,3131641.0,0.031544,1419137165.2125,1619769833.64375,0.21241,0.21241,0.000043,-0.01624,-0.016283,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,10078,012136,2000-03,2000-03-31,SUNW,11,3,3570,93.70313,1586413.0,148651.863573,148695.3465,3521264.0,0.033648,1463410044.721832,1419137165.2125,-0.01624,-0.01624,0.000047,-0.018843,-0.01889,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,10078,012136,2000-04,2000-04-28,SUNW,11,3,3570,91.9375,1588626.0,146054.302875,148651.863573,5067037.0,0.053227,2312568697.088816,1463410044.721832,-0.018843,-0.018843,0.000046,-0.166553,-0.166599,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,10078,012136,2000-05,2000-05-31,SUNW,11,3,3570,76.625,1588626.0,121728.46725,146054.302875,4110327.0,0.04864,1506162044.897727,2312568697.088816,-0.166553,-0.166553,0.00005,0.186786,0.186736,12501.767,6523.737,1586.413,508.135,4004.689,1734.009,786.202,1484.478,<NA>,1.722,5978.03,7344.078,3132.861,3843.826,178.776,4004.689,2002.0,508.135,0.32,3,6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149664,93436,184996,2024-08,2024-08-30,TSLA,11,3,9999,214.11,3194640.0,684004.3704,741380.136746,16108365.0,0.036037,15323491821.250282,32114698553.414902,-0.077391,-0.077391,0.000048,0.221942,0.221894,112832.0,66468.0,3194.0,1400.0,25500.0,19644.0,3505.0,2351.0,9503.0,3012.0,45569.0,52977.0,31094.0,27729.0,1278.0,25500.0,3854.0,1400.0,0.44,2,12,106618.0,62634.0,62634.0,81.0,0.0,0.0,0.0,14997.0,96773.0,74446.0,13558.0,8769.0,6528.0,3045.0,43009.0,49616.0,29637.0,28748.0,4667.0,96773.0,14997.0,13256.0,12
149665,93436,184996,2024-09,2024-09-30,TSLA,11,3,9999,261.63,3207000.0,839047.41,684004.3704,16042065.0,0.033182,18839434038.050682,15323491821.250282,0.221942,0.221942,0.00004,-0.045025,-0.045065,112832.0,66468.0,3194.0,1400.0,25500.0,19644.0,3505.0,2351.0,9503.0,3012.0,45569.0,52977.0,31094.0,27729.0,1278.0,25500.0,3854.0,1400.0,0.44,2,12,106618.0,62634.0,62634.0,81.0,0.0,0.0,0.0,14997.0,96773.0,74446.0,13558.0,8769.0,6528.0,3045.0,43009.0,49616.0,29637.0,28748.0,4667.0,96773.0,14997.0,13256.0,12
149666,93436,184996,2024-10,2024-10-31,TSLA,11,3,9999,249.85001,3210060.0,802033.523101,839047.41,19014312.0,0.053205,20055390329.118835,18839434038.050682,-0.045025,-0.045025,0.000039,0.381469,0.38143,112832.0,66468.0,3194.0,1400.0,

In [9]:
factor_base.columns

Index(['permno', 'gvkey', 'month', 'date', 'ticker', 'shrcd', 'exchcd', 'siccd', 'prc', 'shrout', 'mktcap', 'mktcap_lag1', 'vol', 'volatility_daily', 'avg_dollar_vol',
       'avg_dollar_vol_lag1', 'ret', 'retx', 'rf_1m', 'ret_fwd_1m', 'excess_ret_fwd_1m', 'atq', 'ceqq', 'cshoq', 'niq', 'saleq', 'cogsq', 'oibdpq', 'xsgaq', 'dlttq', 'dlcq',
       'ltq', 'actq', 'cheq', 'lctq', 'dpq', 'revtq', 'oancfy', 'ibq', 'epspxq', 'fqtr', 'fyr', 'datadate', 'at', 'ceq', 'seq', 'txditc', 'pstkrv', 'pstkl', 'pstk', 'ni', 'sale',
       'cogs', 'oibdp', 'xsga', 'dltt', 'dlc', 'lt', 'act', 'che', 'lct', 'dp', 'revt', 'ib', 'oancf', 'fyr', 'datadate'],
      dtype='object')

In [10]:
factor_base.to_csv("C:/Users/woota/OneDrive/Desktop/새 폴더/Personal projects/HMM ML strategy/panel_data.xlsx")

### HMM Data only

In [11]:
# === Fama–French monthly factors (WRDS) ===
# Requires: crsp_m already loaded (used to set start_date)
import pandas as pd
import wrds

# Connect to WRDS
db = wrds.Connection()

# Use earliest CRSP monthly date to bound the FF query
start_date = pd.to_datetime(crsp_m['date'], errors='coerce').min().strftime('%Y-%m-%d')

# Pull FF factors
ff = db.raw_sql(f"""
    select date::date as date,
           mktrf::float8 as mktrf_pct,
           smb::float8   as smb_pct,
           hml::float8   as hml_pct,
           rf::float8    as rf_pct
    from ff.factors_monthly
    where date >= '{start_date}'
""")

# Convert to datetime and decimal returns
ff['date']  = pd.to_datetime(ff['date'], errors='coerce')
ff['mktrf'] = pd.to_numeric(ff['mktrf_pct'], errors='coerce') / 100.0
ff['smb']   = pd.to_numeric(ff['smb_pct'], errors='coerce') / 100.0
ff['hml']   = pd.to_numeric(ff['hml_pct'], errors='coerce') / 100.0
ff['rf_1m'] = pd.to_numeric(ff['rf_pct'], errors='coerce') / 100.0

# Sort and reset index
ff = ff.sort_values('date').reset_index(drop=True)

print("FF factors pulled. Range:", ff['date'].min().date(), "→", ff['date'].max().date())


Enter your WRDS username [woota]: chriswt
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
FF factors pulled. Range: 2000-02-01 → 2025-06-01


In [12]:
import pandas as pd
import numpy as np

# ===============================
# HMM OBSERVABLES (multi-series panel for two HMMs)
# Inputs in memory: crsp_m, crsp_d, sp500, and (preferably) ff
# Output: hmm_obs with columns:
#   date (month-end), mkt_excess_ret, mkt_realized_vol, mkt_ret_12m_ex1m,
#   mkt_ma_slope_6m, drawdown_12m, amihud_median
# ===============================

# ---------- 0) Basic checks ----------
for v in ['crsp_m', 'crsp_d', 'sp500']:
    if v not in globals():
        raise NameError(f"Required DataFrame `{v}` is not in memory.")

# ---------- 1) Normalize CRSP inputs (no tz ops) ----------
crsp_m_ = crsp_m.copy()
crsp_m_['date']   = pd.to_datetime(crsp_m_['date'], errors='coerce')
crsp_m_['month']  = crsp_m_['date'].dt.to_period('M')
crsp_m_['prc']    = pd.to_numeric(crsp_m_['prc'], errors='coerce').abs()
crsp_m_['shrout'] = pd.to_numeric(crsp_m_['shrout'], errors='coerce')  # thousands
crsp_m_['ret']    = pd.to_numeric(crsp_m_['ret'], errors='coerce')
crsp_m_ = crsp_m_.dropna(subset=['date']).sort_values(['permno','month'])

crsp_d_ = crsp_d.copy()
crsp_d_['date']  = pd.to_datetime(crsp_d_['date'], errors='coerce')
crsp_d_['month'] = crsp_d_['date'].dt.to_period('M')
crsp_d_['ret']   = pd.to_numeric(crsp_d_['ret'], errors='coerce')
# Optional but useful for liquidity calc
crsp_d_['prc']   = pd.to_numeric(crsp_d_.get('prc'), errors='coerce').abs()
crsp_d_['vol']   = pd.to_numeric(crsp_d_.get('vol'), errors='coerce')
crsp_d_ = crsp_d_.dropna(subset=['date'])

# PIT S&P 500 membership
sp500_ = sp500.copy()
sp500_['begdt'] = pd.to_datetime(sp500_['begdt'], errors='coerce')
sp500_['enddt'] = pd.to_datetime(sp500_['enddt'], errors='coerce')
sp500_['beg_m'] = sp500_['begdt'].dt.to_period('M')
sp500_['end_m'] = sp500_['enddt'].dt.to_period('M')

# Treat open-ended membership as far-future month
max_month_in_data = pd.concat([crsp_m_['month'], crsp_d_['month']]).dropna().max()
sp500_['end_m'] = sp500_['end_m'].fillna(max_month_in_data)

sp500_keep = sp500_[['permno','beg_m','end_m']].dropna(subset=['beg_m'])

# Restrict CRSP panels to PIT membership windows
crsp_m_ = crsp_m_.merge(sp500_keep, on='permno', how='inner')
crsp_m_ = crsp_m_[(crsp_m_['month'] >= crsp_m_['beg_m']) & (crsp_m_['month'] <= crsp_m_['end_m'])].drop(columns=['beg_m','end_m'])

crsp_d_ = crsp_d_.merge(sp500_keep, on='permno', how='inner')
crsp_d_ = crsp_d_[(crsp_d_['month'] >= crsp_d_['beg_m']) & (crsp_d_['month'] <= crsp_d_['end_m'])].drop(columns=['beg_m','end_m'])

# Build a continuous monthly calendar spine (min..max)
m0, m1 = crsp_m_['month'].min(), crsp_m_['month'].max()
calendar = (
    pd.DataFrame({'month': pd.period_range(m0, m1, freq='M')})
      .assign(date=lambda df: df['month'].dt.to_timestamp('M'))
)

# ---------- 2) Risk-free (RF) from in-memory Fama–French (`ff`) ----------
# We DO NOT connect here. We require `ff` to be loaded previously.
def _rf_from_ff_in_memory():
    if 'ff' not in globals() or not isinstance(ff, pd.DataFrame) or 'date' not in ff.columns:
        return None
    tmp = ff[['date']].copy()
    if 'rf_1m' in ff.columns:
        tmp['rf_1m'] = pd.to_numeric(ff['rf_1m'], errors='coerce')
    elif 'rf_pct' in ff.columns:
        tmp['rf_1m'] = pd.to_numeric(ff['rf_pct'], errors='coerce') / 100.0
    else:
        return None
    tmp['date']  = pd.to_datetime(tmp['date'], errors='coerce')
    tmp['month'] = tmp['date'].dt.to_period('M')
    out = (tmp[['month','rf_1m']]
           .dropna(subset=['month','rf_1m'])
           .drop_duplicates('month'))
    return out if len(out) else None

rf_m = _rf_from_ff_in_memory()
if rf_m is None:
    raise RuntimeError(
        "Fama–French factors (`ff`) not found in memory or missing rf columns. "
        "Please run your connection/pull script first to populate `ff`."
    )

# ---------- 3) Value-weighted monthly market return & excess return ----------
# CRSP: mktcap in $mm = prc ($) * shrout (thousands) / 1000
crsp_m_['mktcap'] = (crsp_m_['prc'] * crsp_m_['shrout']) / 1000.0
crsp_m_['mktcap_lag1'] = crsp_m_.sort_values(['permno','month']).groupby('permno', observed=True)['mktcap'].shift(1)

w = crsp_m_[['permno','month','mktcap_lag1']].dropna().copy()
w = w[w['mktcap_lag1'] > 0]
w['sum_m'] = w.groupby('month', observed=True)['mktcap_lag1'].transform('sum')
w = w[w['sum_m'] > 0]
w['w'] = w['mktcap_lag1'] / w['sum_m']
w = w.drop(columns=['sum_m'])

mret = (
    crsp_m_[['permno','month','ret']]
      .merge(w[['permno','month','w']], on=['permno','month'], how='inner')
      .dropna(subset=['ret','w'])
)
vw_ret = (
    mret.assign(wret=lambda df: df['w'] * df['ret'])
        .groupby('month', as_index=False, observed=True)['wret'].sum()
        .rename(columns={'wret': 'mkt_ret_vw'})
)

vw_ret = calendar[['month']].merge(vw_ret, on='month', how='left')
vw_ret = vw_ret.merge(rf_m, on='month', how='left')
vw_ret['mkt_excess_ret'] = (vw_ret['mkt_ret_vw'].astype('float64')
                            - vw_ret['rf_1m'].astype('float64'))

# ---------- 4) Realized market volatility (3-month) from daily eq-weighted returns ----------
# Equal-weighted daily return across S&P names (PIT-filtered)
daily_eq = (
    crsp_d_.sort_values('date')
           .groupby('date', as_index=False, observed=True)['ret'].mean()
           .rename(columns={'ret': 'eq_ret_daily'})
)

# 63-trading-day rolling (approx 3 months), annualized
daily_eq['rv_63d'] = daily_eq['eq_ret_daily'].rolling(63, min_periods=20).std(ddof=0) * np.sqrt(252.0)
daily_eq['month'] = daily_eq['date'].dt.to_period('M')

# Take end-of-month value (ensure within-group sorted by date)
realized_vol = (
    daily_eq.sort_values(['month','date'])
            .groupby('month', as_index=False, observed=True)
            .agg(mkt_realized_vol=('rv_63d', lambda s: s.iloc[-1] if len(s) else np.nan))
)

# ---------- 5) Trend proxies from monthly value-weighted returns ----------
mkt = vw_ret[['month','mkt_ret_vw']].sort_values('month').copy()
mkt['one_plus'] = 1.0 + mkt['mkt_ret_vw']
mkt['one_plus'] = mkt['one_plus'].where(np.isfinite(mkt['one_plus']), np.nan)

# 12–1 market return: product of previous 11 months (exclude most recent)
def _prod_ex_last(window):
    # window is float array length 12
    prod = np.prod(window[:-1])
    return np.nan if not np.isfinite(prod) else (prod - 1.0)

mkt['mkt_ret_12m_ex1m'] = (
    mkt['one_plus']
      .rolling(12, min_periods=12)
      .apply(_prod_ex_last, raw=True)
)

# Build a level/index from monthly returns (start at 1.0)
mkt['level'] = mkt['one_plus'].fillna(1.0).cumprod()

# 6-month moving average of level
mkt['ma6'] = mkt['level'].rolling(6, min_periods=6).mean()

def _ols_slope(y: np.ndarray) -> float:
    """OLS slope over equally spaced time points (0..n-1)."""
    y = np.asarray(y, dtype=float)
    n = y.size
    if n < 2 or np.any(~np.isfinite(y)):
        return np.nan
    x = np.arange(n, dtype=float)
    x -= x.mean()
    y_c = y - y.mean()
    denom = float(np.dot(x, x))
    if denom == 0:
        return np.nan
    return float(np.dot(x, y_c) / denom)

def _safe_div(a, b):
    if b is None or (isinstance(b, float) and (np.isnan(b) or b == 0.0)):
        return np.nan
    return a / b

# Slope of the 6-month MA over the last 6 observations (dimensionless, normalized by current MA)
mkt['mkt_ma_slope_6m'] = (
    mkt['ma6']
      .rolling(6, min_periods=6)
      .apply(lambda arr: _safe_div(_ols_slope(arr), arr[-1]), raw=True)
)

# Drawdown from trailing 12m max of the level
mkt['rolling_max_12m'] = mkt['level'].rolling(12, min_periods=2).max()
mkt['drawdown_12m'] = mkt['level'] / mkt['rolling_max_12m'] - 1.0

mkt_feats = mkt[['month','mkt_ret_12m_ex1m','mkt_ma_slope_6m','drawdown_12m']]

# ---------- 6) Market-wide Amihud liquidity (median across S&P names per month) ----------
# Dollar volume = price * shares traded; guard against non-positive/inf
crsp_d_['dollar_vol'] = (crsp_d_['vol'] * crsp_d_['prc']).replace([np.inf, -np.inf], np.nan)
ami = crsp_d_[['permno','month','ret','dollar_vol']].dropna()
ami = ami[(ami['dollar_vol'] > 0) & ami['ret'].notna()]
ami['amihud'] = ami['ret'].abs() / ami['dollar_vol']
ami_stock_m = ami.groupby(['permno','month'], as_index=False, observed=True)['amihud'].mean()
ami_mkt = ami_stock_m.groupby('month', as_index=False, observed=True)['amihud'].median().rename(columns={'amihud':'amihud_median'})

# ---------- 7) Assemble final HMM observables ----------
hmm = (
    calendar[['month']]
      .merge(vw_ret[['month','mkt_excess_ret']], on='month', how='left')
      .merge(realized_vol, on='month', how='left')
      .merge(mkt_feats, on='month', how='left')
      .merge(ami_mkt, on='month', how='left')
)

# Month-end timestamp
hmm['date'] = hmm['month'].dt.to_timestamp('M')

# Optional: trim months with no market excess return (incomplete spine)
hmm = hmm[~hmm['mkt_excess_ret'].isna()]

hmm_obs = (
    hmm[['date','mkt_excess_ret','mkt_realized_vol','mkt_ret_12m_ex1m',
         'mkt_ma_slope_6m','drawdown_12m','amihud_median']]
      .sort_values('date')
      .reset_index(drop=True)
)

print("HMM observables shape:", hmm_obs.shape)
print(hmm_obs.tail(5))


HMM observables shape: (299, 7)
          date  mkt_excess_ret  mkt_realized_vol  mkt_ret_12m_ex1m  mkt_ma_slope_6m  drawdown_12m  amihud_median
294 2024-08-31        0.024235          0.125420          0.240715         0.022622           0.0            0.0
295 2024-09-30        0.022543          0.133100          0.334640         0.020531           0.0            0.0
296 2024-10-31       -0.008729          0.119599          0.393982         0.019497      -0.00869            0.0
297 2024-11-30        0.060213          0.106674          0.266834         0.019397           0.0            0.0
298 2024-12-31       -0.023449          0.115881          0.284991         0.018839     -0.023412            0.0


In [13]:
hmm_obs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              299 non-null    datetime64[ns]
 1   mkt_excess_ret    299 non-null    float64       
 2   mkt_realized_vol  299 non-null    float64       
 3   mkt_ret_12m_ex1m  288 non-null    float64       
 4   mkt_ma_slope_6m   290 non-null    float64       
 5   drawdown_12m      299 non-null    Float64       
 6   amihud_median     299 non-null    Float64       
dtypes: Float64(2), datetime64[ns](1), float64(4)
memory usage: 17.1 KB


In [14]:
zero_summary = pd.DataFrame({
    'zero_count': (hmm_obs.fillna(0) == 0).sum(),
    'zero_pct': (hmm_obs.fillna(0) == 0).sum() / len(hmm_obs) * 100
})

print(zero_summary)

                  zero_count   zero_pct
date                       0        0.0
mkt_excess_ret             0        0.0
mkt_realized_vol           0        0.0
mkt_ret_12m_ex1m          11    3.67893
mkt_ma_slope_6m            9   3.010033
drawdown_12m             124  41.471572
amihud_median              0        0.0


In [15]:
hmm.to_csv("C:/Users/woota/OneDrive/Desktop/새 폴더/Personal projects/HMM ML strategy/regime_data.csv")